In [ ]:
import pandas as pd
# import numpy as np
# from datetime import time, datetime
# from matplotlib import pyplot as plt
# from matplotlib.ticker import MultipleLocator
# import matplotlib.ticker as ticker
# from matplotlib.dates import DateFormatter
import os

In [ ]:
workout_type = 'A'

In [ ]:
FILENAME = 'raw_data/CSVs/erg scores - 2023-03-15.csv'
a = pd.read_csv(FILENAME)

In [ ]:
def split_to_seconds(duration_string: str):
    return float(duration_string.split(':')[0])*60+float(duration_string.split(':')[1])

a["split_in_s"] = a["/500m"].apply(lambda x: split_to_seconds(x))

In [ ]:
# from datetime import datetime, timedelta

# def convert_str_duration_to_time_delta(duration_string: str):
#     dt = datetime.strptime(duration_string, "%M:%S.%f")
#     return timedelta(minutes=dt.minute, seconds=dt.second, microseconds=dt.microsecond)

# a["/500m_duration"] = a["/500m"].apply(lambda x: convert_str_duration_to_time_delta(x))

In [ ]:
a['cutoff_minute'] = a['time'].str.split(':').str[0].astype(int)

In [ ]:
a['workout_date'] = FILENAME[-14:-4]
a['workout_type'] = workout_type
a['workout_date'] = pd.to_datetime(a['workout_date']).dt.tz_localize('Europe/London')

In [ ]:
import pandera as pa
from pandera import Column, DataFrameSchema, Check

In [ ]:
allowed_workout_types = ['A', 'B']

schema = pa.DataFrameSchema({
    'workout_date': pa.Column(pa.engines.pandas_engine.DateTime(tz='Europe/London')),
    'workout_type': pa.Column(pa.Category, checks = pa.Check.isin(allowed_workout_types), coerce=True, regex = True),
    'cutoff_minute': pa.Column(int, coerce=True),
    'split_in_s': pa.Column(float, coerce=True),
    'meter': pa.Column(int, coerce=True)
})

In [ ]:
schema.validate(a)

In [ ]:
OUTPUT_FILEPATH = f'processed_data/workout_{workout_type}.csv'
if os.path.isfile(OUTPUT_FILEPATH):
    output = pd.read_csv(OUTPUT_FILEPATH)
    output = pd.concat([output, a], ignore_index=True)
    output.to_csv(OUTPUT_FILEPATH, index=False)
else:
    a.to_csv(OUTPUT_FILEPATH, index=False)

In [5]:

# date='2023-03-01'

import os

import boto3
import pandas as pd

boto3.setup_default_session(profile_name='dev-profile')
client = boto3.client("s3")


for key in client.list_objects(Bucket=bucket)['Contents']:
    print(key['Key'])

# result = client.get_bucket_policy(Bucket=bucket)
# print(result['Policy'])


processed-data/
raw-data/
raw-data/manual-input/
raw-data/manual-input/2023-01-17.csv
raw-data/manual-input/2023-03-01.csv
raw-data/manual-input/2023-03-15.csv


In [9]:
import io

In [10]:
bucket = 'ergscorepipeline'
key = 'raw-data/manual-input/2023-03-15.csv'

obj = client.get_object(Bucket=bucket, Key=key)
df = pd.read_csv(io.BytesIO(obj['Body'].read()))

In [11]:
df

,Unnamed: 0,cutoff_minute,metres,/500m,s/m
0,0,6,1209,2:28.8,20
1,1,12,1218,2:27.7,20
2,2,18,1227,2:26.6,20
3,3,24,1236,2:25.6,20
4,4,30,1252,2:23.7,20
